In [ ]:
import pandas as pd
import random

In [ ]:
# Sample subreddit topics
subreddits = ['politics', 'gaming', 'relationships', 'news', 'funny']

In [ ]:
# Clean comments
clean_comments = {
    'politics': [
        "I appreciate the different point of view.",
        "Interesting how policies evolve over time.",
        "Thanks for sharing this article.",
        "This debate was surprisingly civil.",
        "Great breakdown of the election results."
    ],
    'gaming': [
        "This game has amazing graphics!",
        "Loved the new update.",
        "Nice strategy, I'll try it out.",
        "Great team coordination in this match.",
        "I’ve been waiting for this release!"
    ],
    'relationships': [
        "You gave great advice here.",
        "Glad things worked out for you.",
        "Communication is definitely key.",
        "Thanks for being open about your journey.",
        "That was a really thoughtful response."
    ],
    'news': [
        "Very informative report.",
        "This gives a new perspective on the issue.",
        "Interesting piece, thanks for posting.",
        "The facts were laid out well.",
        "It’s good to stay informed."
    ],
    'funny': [
        "This made me laugh so hard!",
        "10/10 comedy!",
        "Perfect timing 😂",
        "That’s actually hilarious!",
        "I can't stop watching this lol"
    ]
}

In [ ]:
# Rule-breaking comments
toxic_comments = {
    'politics': [
        "You’re just a brain-dead sheep.",
        "Only an idiot would believe that.",
        "Go cry somewhere else, clown.",
        "You're what's wrong with this country.",
        "This post is absolute garbage."
    ],
    'gaming': [
        "Trash gameplay like always.",
        "You suck. Uninstall the game.",
        "Only noobs use that weapon.",
        "Typical loser move.",
        "You're the worst teammate ever."
    ],
    'relationships': [
        "That’s the dumbest advice ever.",
        "No wonder you got dumped.",
        "You're pathetic.",
        "This is why no one likes you.",
        "Go fix your own mess first."
    ],
    'news': [
        "Fake news, shut up.",
        "You sound like a government puppet.",
        "Go educate yourself, moron.",
        "Stop spreading BS propaganda.",
        "This post is complete trash."
    ],
    'funny': [
        "That wasn’t funny, idiot.",
        "You must have zero brain cells.",
        "Lame. Try harder next time.",
        "Please delete your account.",
        "This joke is so bad it hurts."
    ]
}

In [ ]:
# Generate dataset
comments = []
labels = []
subs = []

In [ ]:
for sub in subreddits:
    for _ in range(40):  # 20 clean + 20 toxic per subreddit
        comments.append(random.choice(clean_comments[sub]))
        labels.append(0)
        subs.append(sub)
        comments.append(random.choice(toxic_comments[sub]))
        labels.append(1)
        subs.append(sub)

In [ ]:
# Shuffle dataset
data = list(zip(comments, labels, subs))
random.shuffle(data)
comments, labels, subs = zip(*data)

df = pd.DataFrame({
    "comment": comments,
    "label": labels,
    "subreddit": subs
})

In [ ]:
df.head()

,comment,label,subreddit
0,This game has amazing graphics!,0,gaming
1,That’s the dumbest advice ever.,1,relationships
2,This joke is so bad it hurts.,1,funny
3,This gives a new perspective on the issue.,0,news
4,Interesting how policies evolve over time.,0,politics


In [ ]:
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = "".join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

df["cleaned"] = df["comment"].apply(clean_text)

In [ ]:
# One-hot encode the subreddit column
df_encoded = pd.get_dummies(df, columns=["subreddit"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

In [ ]:
# TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_text = vectorizer.fit_transform(df_encoded["cleaned"])

In [ ]:
# Optional: Add subreddit features
subreddit_features = df_encoded.drop(columns=["comment", "label", "cleaned"])
X_full = hstack([X_text, subreddit_features.values])

In [ ]:
# Labels
y = df_encoded["label"]

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        41

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [21]:
import pandas as pd

In [24]:
# Example: Generate sample submission
# Replace this with your actual test data and predictions if available
submission = pd.DataFrame({
    'comment_id': ['c1', 'c2', 'c3', 'c4'],
    'rule_violation': [0, 1, 0, 1]
})

In [25]:
# Save as CSV
submission.to_csv('submission.csv', index=False)

In [26]:
# Confirm it's saved
submission.head()

,comment_id,rule_violation
0,c1,0
1,c2,1
2,c3,0
3,c4,1


# 🛡️ Reddit Rule Violation Classifier
This project builds a machine learning model to detect whether Reddit comments break subreddit rules. Inspired by real-world moderation challenges, the model is trained on a synthetic but realistic dataset.

## 📘 Project Overview
This project uses a synthetic dataset that mimics real Reddit comments to train a binary classifier that detects whether a comment violates subreddit rules. It demonstrates skills in:
- Text preprocessing (NLP)
- Feature engineering
- Model training with scikit-learn
- Evaluation and performance metrics

This project was originally inspired by a Kaggle competition but is now repurposed for portfolio demonstration.

## 🧪 Generate a Realistic Dataset

Since we are not using real Reddit data, we generate a synthetic dataset that simulates Reddit comments and labels:
- 0 → Safe comment
- 1 → Rule-breaking comment

We also include subreddit names to simulate community context.

## 🧼 Text Preprocessing

We clean the comment text using NLP techniques:
- Lowercasing
- Removing punctuation
- Tokenizing
- Removing stopwords

This helps the model better understand the patterns in the text.

## 📊 TF-IDF Vectorization and Splitting

We transform the cleaned text into numerical features using TF-IDF (Term Frequency–Inverse Document Frequency). Then, we split the data into training and testing sets to evaluate performance.

## 🤖 Train and Evaluate the Model

We use a simple but effective model (e.g., Logistic Regression or Naive Bayes) to classify the comments. Then, we evaluate the model using accuracy, precision, recall, and F1 score.